# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в заголовке которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# определяем список хабов, которые нам интересны
KEYWORDS  =  ['парсинг','rust' ]

In [3]:
def make_headers():
    headers = {
        'user-agent': 'Mozilla/5.0', 
    }
    return headers

def make_parameters():
    params = {
 
    }
    return params


def get_html(url):
  try:
    response = requests.get(url, headers=make_headers(), params=make_parameters())
    if response.status_code == 200:
            return response.text
    else:
             return '<ошибка на сервере>'
  except requests.ConnectionError:
        return '<сетевая ошибка>'


response = get_html('https://habr.com/ru/all/')
soup = BeautifulSoup(response, 'lxml')
posts = soup.find_all('article', class_="tm-articles-list__item")

data_body = []
data_header = ['date','header','link']
for post in posts:
  title =  post.find_all('a', class_="tm-article-snippet__title-link")
  if title:
    title = title[0].text.lower()
    if any([keyword in title for keyword in  KEYWORDS]):
      time = post.time.attrs.get('datetime')
      post_link= post.find_all('a',class_="tm-article-snippet__title-link")[0].attrs.get('href')
      post_link = 'https://habr.com' + post_link
      data_body.append([time,title,post_link])

df_habr_articles = pd.DataFrame(data_body, columns = data_header)
df_habr_articles['date'] = pd.to_datetime(df_habr_articles['date'])
df_habr_articles

,date,header,link
0,2022-01-17 19:35:39+00:00,связный список на rust в стиле с/c++,https://habr.com/ru/post/646105/


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только заголовки статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [4]:
def check_text_body(post_link):  
  response = get_html(post_link) 
  soup = BeautifulSoup(response, 'lxml')
  
  text = ''
  for item in soup.find_all('div', class_="article-formatted-body"):
    text += item.text
    text = text.lower()
    if any([keyword in text for keyword in  KEYWORDS]):
      return text.strip()
    else:
      return None  
  


In [5]:
response = get_html('https://habr.com/ru/all/')
soup = BeautifulSoup(response, 'lxml')
posts = soup.find_all('article', class_="tm-articles-list__item")

data_body = []
data_header = ['date','header','link', 'text']
for post in posts:
  title =  post.find_all('a', class_="tm-article-snippet__title-link")
  if title:
    title = title[0].text.lower()
    post_link= post.find_all('a',class_="tm-article-snippet__title-link")[0].attrs.get('href')
    post_link = 'https://habr.com' + post_link
    time = post.time.attrs.get('datetime')
    text = check_text_body(post_link)
    flag = not(text is None) 
    print(title , '---->', flag)
    if any([keyword in title for keyword in  KEYWORDS]) or check_text_body(post_link):
      data_body.append([time,title,post_link, text])

df_habr_articles = pd.DataFrame(data_body, columns = data_header)
df_habr_articles['date'] = pd.to_datetime(df_habr_articles['date'])
df_habr_articles


продуктовые vs проектные команды ----> False
6 грейдов в карьере инженера по автоматизации: основные критерии развития ----> False
hackthebox разбор id exposed. на поиски сары. продолжаем разбор лаборатории osint (уровень: easy) ----> False
связный список на rust в стиле с/c++ ----> True
дорога на запад. геодезия и отвага ----> False
мини-пк в 2022 году: интересные модели, которые могут пригодиться в офисе и дома ----> False
как своими руками сделать wifi камеру слежения (esp32-cam) ----> False
адаптивное свойство одной строкой ----> False
делаем асинхронность асинхронной, разбираемся в планировщике go, ругаем linux ----> False
как прошла плисовая конференция в питере? ----> False
лучшие практики тестирования микросервисов ----> False
вышел релиз gitlab 14.6 с улучшенной работой geo и поддержкой .net 6 в sast ----> False
реверс-инжинирим структуру бд postgresql по плану запроса к ней ----> False
security week 2203: wormable-уязвимость в windows ----> False
hsluv — удобное цветовое прос

,date,header,link,text
0,2022-01-17 19:35:39+00:00,связный список на rust в стиле с/c++,https://habr.com/ru/post/646105/,может ли безопасный и стабильный rust противоп...


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы. Данные (почтовые адреса) передаются на сервис в json-формате (параметр `json` функции `post`).

In [6]:
EMAIL = ["124@mail.ru"]

url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
request_body = {"emailAddresses":EMAIL}



def make_headers():
    headers = {
         'user-agent': 'Mozilla/5.0', 
         'Host': 'identityprotection.avast.com',
          'Accept': 'application/json, text/plain, */*',
          'Accept-Language': 'en-US,en;q=0.5',
          'Accept-Encoding': 'gzip, deflate, br',
          'Referer': 'https://www.avast.com/hackcheck/',
          'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
          'Vaar-Header-App-Build-Version': '1.0.0',
          'Vaar-Version': '0',
          'Content-Type': 'application/json;charset=utf-8',
          'Content-Length': '34',
          'Origin': 'https://www.avast.com',
          'Connection': 'keep-alive',
          'Pragma': 'no-cache',
          'Cache-Control': 'no-cache',
    }
    return headers


r = requests.post(url,headers=make_headers(), json=request_body)
if r.status_code == 200:
  verified_emails = r.json()

In [7]:
df_breaches = pd.DataFrame.from_dict(verified_emails['breaches'], orient='index').reset_index()
df_breaches[['publishDate','site','description']]

,publishDate,site,description
0,2020-07-23T00:00:00Z,wattpad.com,"In June 2020, the online writing community Wat..."
1,2020-12-10T00:00:00Z,discerner-les-temps.com,"In November 2020, a collection of over 23,000 ..."
2,2020-12-10T00:00:00Z,ctpnrlyk.fun,"In November 2020, a collection of over 23,000 ..."
3,2020-12-10T00:00:00Z,173.212.232.240,"In November 2020, a collection of over 23,000 ..."
4,2019-08-15T00:00:00Z,simba-trade.ru,"At an unconfirmed date, children toy retailer ..."
5,2019-10-17T00:00:00Z,zynga.com,"In September 2019, the game developer Zynga wa..."
6,2020-01-16T00:00:00Z,proshkolu.ru,"In November 2019, the Russian education portal..."
7,2020-11-05T00:00:00Z,vimeworld.ru,"In January 2018, the Russian Minecraft server ..."
8,2020-10-22T00:00:00Z,ozon.ru,"At an unconfirmed date, the Russian online sto..."
9,2019-05-23T00:00:00Z,livejournal.com,"In 2017, social network LiveJournal's database..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

In [8]:
def make_headers():
    headers = {
        'user-agent': 'Mozilla/5.0', 
    }
    return headers

def make_parameters():
    params = {
        'access_token' : '' ,
        'v' : 5.131,
        'domain' : 'netology',
        'count' : 50
    }
    return params


def get_html(url):
  try:
    response = requests.get(url, headers=make_headers(), params=make_parameters())
    if response.status_code == 200:
            return response
    else:
             return '<ошибка на сервере>'
  except requests.ConnectionError:
        return '<сетевая ошибка>'

In [9]:
url = f'https://api.vk.com/method/wall.get'
response = get_html(url)
response_dict = response.json()

In [10]:
text = []
date = []

for i in range(50):
  text.append(response_dict['response']['items'][i]['text'])
  date.append(response_dict['response']['items'][i]['date'])


df = pd.DataFrame(list(zip(date,text)), columns = ['date', 'text'])
df['date'] = pd.to_datetime(df['date'],unit='s')
df

,date,text
0,2022-01-14 14:00:00,"Поздравляем, вы добрались до конца первой рабо..."
1,2022-01-17 15:51:00,"С цветами всё просто: необходимы полив, тепло ..."
2,2022-01-17 07:44:00,Напоминаем про бесплатный курс «Основы Figma» ...
3,2022-01-16 09:01:00,К концу напряжённого рабочего дня силы обычно ...
4,2022-01-15 09:11:00,Зачитались статьями из подборки и ушли играть ...
5,2022-01-14 07:27:00,"По данным сервиса по трудоустройству Trud.com,..."
6,2022-01-13 15:31:00,Дизайнер — одна из самых популярных диджитал-п...
7,2022-01-13 07:39:00,Добиться новых вершин в карьере в 2022 году по...
8,2022-01-12 13:30:00,"Длинные выходные позади, пора восстанавливать ..."
9,2022-01-12 07:16:00,18 января стартует бесплатный курс «Как стать ...


#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой [Google Colab](https://colab.research.google.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
